In [1]:
import gc
import torch
import time

from pydantic import BaseModel, Field
from typing import Literal, List, Dict, Any, Tuple
from typing_extensions import TypedDict
from IPython.display import Image, display

from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_classic.storage import InMemoryStore
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import CrossEncoderReranker
from langchain_classic.retrievers.parent_document_retriever import ParentDocumentRetriever
from langgraph.graph import END, StateGraph

In [2]:
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'}, encode_kwargs={'batch_size':8})
model = ChatOpenAI(model='gpt-5-mini', temperature=0)
hf_model = HuggingFaceCrossEncoder(model_name='BAAI/bge-reranker-v2-m3', model_kwargs={'device':'cuda'})

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [3]:
loader = PyPDFLoader('./data/투자설명서.pdf')
docs = loader.load()
full_text = '\n\n'.join(doc.page_content for doc in docs)

In [4]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=300)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=60)
vectorstore = Chroma(collection_name='split_parents_self_rag', embedding_function=embeddings)
docstore = InMemoryStore()

In [5]:
retriever = ParentDocumentRetriever(
    docstore=docstore,
    vectorstore=vectorstore,
    parent_splitter=parent_splitter,
    child_splitter=child_splitter
)

docs = retriever.add_documents([Document(page_content=full_text)])

In [6]:
print(vectorstore._collection.count())
print(len(list(docstore.yield_keys())))

1323
264


In [7]:
reranker = CrossEncoderReranker(model=hf_model, top_n=5)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=retriever
)

In [8]:
class GraphState(TypedDict):
    question: str
    generation: str
    documents: List[Document]
    retry_count: int

In [9]:
def retrieve(state):
    print('--- retrieve ---')

    question = state['question']
    documents = compression_retriever.invoke(question)

    return {'documents':documents, 'question':question, 'retry_count':0}

In [10]:
def grade_documents(state):
    print('--- check relevance ---')

    question = state['question']
    documents = state['documents']

    class Grade(BaseModel):
        binary_score: str = Field(description='문서와 질문이 관련이 있으면 "yes", 아니면 "no"')
    
    structured_llm_grader = model.with_structured_output(Grade)

    system = '''당신은 제공된 연관 문서가 주어진 질문과 관련이 있는지, 그리고 질문에 답하는 데 유용한 정보를 제공하는지 판단하는 것입니다.
    철저하게 검증하여 문서가 질문의 키워드나 의미를 포함하고 있다면 "yes"를, 아니라면 "no"를 출력하세요.'''
    grade_prompt = ChatPromptTemplate.from_messages(
        [('system', system),
         ('human', '질문: {question}\n문서: {documents}')]
    )
    retrieval_grader = grade_prompt | structured_llm_grader

    filtered_docs = []
    for doc in documents:
        score = retrieval_grader.invoke({'question':question, 'documents':doc.page_content})
        if score.binary_score == 'yes':
            print(f' -- 문서 채택')
            filtered_docs.append(doc)
        else:
            print(f' -- 문서 기각')
    
    return {'documents':filtered_docs, 'question':question}

In [11]:
def transform_query(state):
    print('--- transform query ---')

    question = state['question']
    documents = state['documents']
    retry_count = state.get('retry_count', 0) + 1

    system = '''당신은 사용자의 질문을 검색에 더 최적화된 형태로 다듬는 전문가입니다.
    원래 질문의 의도를 유지하면서, 더 좋은 문서를 찾을 수 있도록 질문을 수정하세요.'''
    retry_prompt = ChatPromptTemplate.from_messages(
        [('system', system),
         ('human', '원본 질문: {question}')]
    )
    question_rewriter = retry_prompt | model | StrOutputParser()

    better_question = question_rewriter.invoke({'question':question})
    print(f' -- 수정된 질문: {better_question}')

    return {'documents':documents, 'question':better_question, 'retry_count':retry_count}

In [12]:
def generate(state):
    print('--- generate ---')

    question = state['question']
    documents = state['documents']

    context = '\n\n'.join(doc.page_content for doc in documents)
    prompt = ChatPromptTemplate.from_template(
        '''다음 문서들을 바탕으로 질문에 답변하세요.
        문서: {context}
        질문: {question}
        답변:'''
    )
    rag_chain = prompt | model | StrOutputParser()

    generation = rag_chain.invoke({'context':context, 'question':question})

    return {'documents':documents, 'question':question, 'generation':generation}